## 基线测试

- 输入特征：
    - Dates中的小时字段
    - DayOfWeek
    - PdDistrict
    - Address中是否包含"Block"字段
    - 经度
    - 纬度

### 读取数据集

In [1]:
import pandas as pd

train_data = pd.read_csv('../datasets/train.csv')

### 随机模型

In [42]:
from sklearn.metrics import log_loss
from sklearn.preprocessing import OneHotEncoder
import numpy as np

# 将预测类型按字母顺序排序
categories = train_data['Category'].unique()
categories.sort()
print('categories: ', categories)

# 将预测标签进行独热编码
encoder = OneHotEncoder(sparse=False)
encoder.fit(categories.reshape(len(categories), 1))

y_true = np.array(train_data['Category'])
y_true = encoder.transform(y_true.reshape(len(y_true), 1))

# 训练集样本个数
sample_cnt = train_data.shape[0]

# 随机预测
import random

# 随机获取样本个数个数字，数字范围[0, 38]
y_pred = random.choices(range(len(categories)), k=sample_cnt)


# 独热编码
y_pred = encoder.transform(y_pred)



categories:  ['ARSON' 'ASSAULT' 'BAD CHECKS' 'BRIBERY' 'BURGLARY' 'DISORDERLY CONDUCT'
 'DRIVING UNDER THE INFLUENCE' 'DRUG/NARCOTIC' 'DRUNKENNESS'
 'EMBEZZLEMENT' 'EXTORTION' 'FAMILY OFFENSES' 'FORGERY/COUNTERFEITING'
 'FRAUD' 'GAMBLING' 'KIDNAPPING' 'LARCENY/THEFT' 'LIQUOR LAWS' 'LOITERING'
 'MISSING PERSON' 'NON-CRIMINAL' 'OTHER OFFENSES'
 'PORNOGRAPHY/OBSCENE MAT' 'PROSTITUTION' 'RECOVERED VEHICLE' 'ROBBERY'
 'RUNAWAY' 'SECONDARY CODES' 'SEX OFFENSES FORCIBLE'
 'SEX OFFENSES NON FORCIBLE' 'STOLEN PROPERTY' 'SUICIDE' 'SUSPICIOUS OCC'
 'TREA' 'TRESPASS' 'VANDALISM' 'VEHICLE THEFT' 'WARRANTS' 'WEAPON LAWS']


d:\Anaconda3\envs\gym\lib\site-packages\numpy\lib\arraysetops.py:518: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask &= (ar1 != a)
d:\Anaconda3\envs\gym\lib\site-packages\numpy\lib\arraysetops.py:522: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


ValueError: Found unknown categories [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38] in column 0 during transform

In [7]:
log_loss(np.array([[0,1,0,0],
                  [1,0,0,0]]),
        np.array([[0,1,0,0],
                 [0,1,0,0]]))

17.269388197455342